# Covid data dashboard in Brazil with Python and Plotly

#### To analyze population, geographic and temporal variations in the distribution of confirmed cases of COVID-19, as part of the set of epidemiological surveillance actions for the disease.

#### We will create a dashboard with plotly and dash, where we will show an interactive map of Brazil, in which we will be able to select the states and visualize the data.

We will start by getting the data from the website https://covid.saude.gov.br/ , from the button "Arquivo CSV"

Then we will create a notebook to work with python and do the library imports and installs:

In [1]:
#--------------------------------------------------
#Installs
#--------------------------------------------------
#!pip install plotly
#!pip install dash
#!pip install json
#!pip install dash_bootstrap_components
#--------------------------------------------------
#Imports 
#--------------------------------------------------
#Graphics Creation
import plotly.express as px  
import plotly.graph_objects as go
#Styles, Interactive dashboard components, placing html code
import dash 
import dash_core_components as dcc 
import dash_html_components as html 
from dash.dependencies import Input, Output  
import dash_bootstrap_components as dbc
#Others libraries
import numpy as np
import pandas as pd
import json

C:\Users\renat\AppData\Local\Temp/ipykernel_11000/3443395610.py:16: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\renat\AppData\Local\Temp/ipykernel_11000/3443395610.py:17: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


#### We will import the downloaded csv file:

In [2]:
df = pd.read_csv("HIST_PAINEL_COVIDBR_13mai2021.csv", sep=";")
df.head(50)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN
5,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-01,10,210147125.0,2,0,0,0,NaN,NaN,NaN
6,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-02,10,210147125.0,2,0,0,0,NaN,NaN,NaN
7,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-03,10,210147125.0,2,0,0,0,NaN,NaN,NaN
8,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-04,10,210147125.0,3,1,0,0,NaN,NaN,NaN
9,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-05,10,210147125.0,7,4,0,0,NaN,NaN,NaN


#### Now we will separate the dataset to facilitate data manipulation. We will separate between Brazil and states and export to csv:

In [3]:
df_states = df[~df['estado'].isna()&df['codmun'].isna()]
df_brasil = df[df['regiao']=='Brasil']
#Exporting to csv 
df_brasil.to_csv('df_brasil.csv')
df_states.to_csv('df_states.csv')

#### And we will use a json file to create the map and identify the data that relates to our database. In this case the column “ID”.

In [5]:
#Reading json file using "r" method (read)
df_states = pd.read_csv("df_states.csv")
df_brasil = pd.read_csv("df_brasil.csv")
token = open(".mapbox_token").read()
brazil_states = json.load(open("geojson/brazil_geo.json", "r"))
brazil_states["features"][0].keys()
df_states_ = df_states[df_states["data"] == "2020-05-13"]
select_columns = {"casosAcumulado": "Casos Acumulados", 
                "casosNovos": "Novos Casos", 
                "obitosAcumulado": "Óbitos Totais",
                "obitosNovos": "Óbitos por dia"}

#### Here we will use an external style sheet using the themes available on bootswatch.com . We chose the LUX theme:

In [6]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

#### In the next steps, we will create the “figs” that will contain our dashboard.
- choropleth does the division by color
- mapbox is an external API that creates more stylized graphics.

We will also pass a series of parameters, the main ones:
- df

- locations: which is how you will split the map by colors.

- color: What information will be the parameter for changing colors.

- center: Indicates where you want the graph to start centered.

- geojson: Which file will use for the json.

- color_continuous_scale: Color palette.

- opacity: Transparency Level.

- hover_data: what will show when hovering the cursor, uses dictionary.

In [ ]:
#--------------------------------------------------
#fig -MAP (example using px.)
#--------------------------------------------------
fig = px.choropleth_mapbox(df_states_, locations="estado",
    geojson=brazil_states, center={"lat": -16.95, "lon": -47.78},  # https://www.google.com/maps/ -> right click -> get lat/lon
    zoom=4, color="casosNovos", color_continuous_scale="Redor", opacity=0.4,
    hover_data={"casosAcumulado": True, "casosNovos": True, "obitosNovos": True, "estado": True}
    )
#Style
fig.update_layout(
                # mapbox_accesstoken=token,
                paper_bgcolor="#D3D3D3",
                mapbox_style="carto-darkmatter",
                autosize=True,
                margin=go.layout.Margin(l=0, r=0, t=0, b=0),
                showlegend=False,)
df_data = df_states[df_states["estado"] == "RO"]
#--------------------------------------------------
#fig2 - Bar Chart (example using go.figure)
#--------------------------------------------------
fig2 = go.Figure(layout={"template":"plotly_white"})
fig2.add_trace(go.Scatter(x=df_data["data"], y=df_data["casosAcumulado"]))
fig2.update_layout(
    paper_bgcolor="#D3D3D3",
    plot_bgcolor="#D3D3D3",
    autosize=True,
    margin=dict(l=10, r=10, b=10, t=10),
    )

## Layout Creation — See dash-bootstrap-components documentation

#### The structure is a big container with rows and columns.
We will do each row by subdividing into columns:

In [ ]:
app.layout = dbc.Container(
    children=[
        dbc.Row([
            
            dbc.Col(
                [
                    dcc.Loading(
                        id="loading-1",
                        type="default",
                        children=[dcc.Graph(id="choropleth-map", figure=fig, 
                            style={'height': '100vh', 'margin-right': '10px'})],
                    ),
                ], md=7),
            
            dbc.Col([
                    html.Div([
                        html.Img(id="logo", src=app.get_asset_url("logosite.png"), height=50),
                        html.H5(children="Evolução COVID-19"),
                        dbc.Button("BRASIL", color="primary", id="location-button", size="lg")
                    ], style={"background-color": "#D3D3D3", "margin": "-25px", "padding": "25px"}),
                    html.P("Informe a data na qual deseja obter informações:", style={"margin-top": "40px","color": "#000000"}),
                    html.Div(
                            className="div-for-dropdown",
                            id="div-test",
                            children=[
                                dcc.DatePickerSingle(
                                    id="date-picker",
                                    min_date_allowed=df_states.groupby("estado")["data"].min().max(),
                                    max_date_allowed=df_states.groupby("estado")["data"].max().min(),
                                    initial_visible_month=df_states.groupby("estado")["data"].min().max(),
                                    date=df_states.groupby("estado")["data"].max().min(),
                                    display_format="MMMM D, YYYY",
                                    style={"border": "0px solid black"},
                                )
                            ],
                        ),
dbc.Row([
                        dbc.Col([dbc.Card([   
                                dbc.CardBody([
                                    html.Span("Casos recuperados", className="card-text"),
                                    html.H3(style={"color": "#D3D3D3"}, id="casos-recuperados-text"),
                                    html.Span("Em acompanhamento", className="card-text"),
                                    html.H5(id="em-acompanhamento-text"),
                                    ])
                                ], color="light", outline=True, style={"margin-top": "10px",
                                        "box-shadow": "0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)",
                                        "color": "#000000"})], md=4),
                        dbc.Col([dbc.Card([   
                                dbc.CardBody([
                                    html.Span("Casos confirmados totais", className="card-text"),
                                    html.H3(style={"color": "#389fd6"}, id="casos-confirmados-text"),
                                    html.Span("Novos casos na data", className="card-text"),
                                    html.H5(id="novos-casos-text"),
                                    ])
                                ], color="light", outline=True, style={"margin-top": "10px",
                                        "box-shadow": "0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)",
                                        "color": "#000000"})], md=4),
                        dbc.Col([dbc.Card([   
                                dbc.CardBody([
                                    html.Span("Óbitos confirmados", className="card-text"),
                                    html.H3(style={"color": "#DF2935"}, id="obitos-text"),
                                    html.Span("Óbitos na data", className="card-text"),
                                    html.H5(id="obitos-na-data-text"),
                                    ])
                                ], color="light", outline=True, style={"margin-top": "10px",
                                        "box-shadow": "0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)",
                                        "color": "#000000"})], md=4),
                    ]),
html.Div([
                        html.P("Selecione que tipo de dado deseja visualizar:", style={"margin-top": "25px", "color": "#000000"}),
                        dcc.Dropdown(
                                        id="location-dropdown",
                                        options=[{"label": j, "value": i}
                                            for i, j in select_columns.items()
                                        ],
                                        value="casosNovos",
                                        style={"margin-top": "10px"}
                                    ),
                        dcc.Graph(id="line-graph", figure=fig2, style={
                            "background-color": "#000000",
                            }),
                        ], id="teste")
                ], md=5, style={
                          "padding": "25px",
                          "background-color": "#D3D3D3"
                          }),
], class_name='g-0')
    ], fluid=True, 
)

#### Here we will do the interactivity step, where the data will be updated according to the selection:

In [ ]:
@app.callback(
    [
        Output("casos-recuperados-text", "children"),
        Output("em-acompanhamento-text", "children"),
        Output("casos-confirmados-text", "children"),
        Output("novos-casos-text", "children"),
        Output("obitos-text", "children"),
        Output("obitos-na-data-text", "children"),
    ], [Input("date-picker", "date"), Input("location-button", "children")]
)
def display_status(date, location):
    # print(location, date)
    if location == "BRASIL":
        df_data_on_date = df_brasil[df_brasil["data"] == date]
    else:
        df_data_on_date = df_states[(df_states["estado"] == location) & (df_states["data"] == date)]

    recuperados_novos = "-" if df_data_on_date["Recuperadosnovos"].isna().values[0] else f'{int(df_data_on_date["Recuperadosnovos"].values[0]):,}'.replace(",", ".") 
    acompanhamentos_novos = "-" if df_data_on_date["emAcompanhamentoNovos"].isna().values[0]  else f'{int(df_data_on_date["emAcompanhamentoNovos"].values[0]):,}'.replace(",", ".") 
    casos_acumulados = "-" if df_data_on_date["casosAcumulado"].isna().values[0]  else f'{int(df_data_on_date["casosAcumulado"].values[0]):,}'.replace(",", ".") 
    casos_novos = "-" if df_data_on_date["casosNovos"].isna().values[0]  else f'{int(df_data_on_date["casosNovos"].values[0]):,}'.replace(",", ".") 
    obitos_acumulado = "-" if df_data_on_date["obitosAcumulado"].isna().values[0]  else f'{int(df_data_on_date["obitosAcumulado"].values[0]):,}'.replace(",", ".") 
    obitos_novos = "-" if df_data_on_date["obitosNovos"].isna().values[0]  else f'{int(df_data_on_date["obitosNovos"].values[0]):,}'.replace(",", ".") 
    return (
            recuperados_novos, 
            acompanhamentos_novos, 
            casos_acumulados, 
            casos_novos, 
            obitos_acumulado, 
            obitos_novos,
            )


@app.callback(
        Output("line-graph", "figure"),
        [Input("location-dropdown", "value"), Input("location-button", "children")]
)
def plot_line_graph(plot_type, location):
    if location == "BRASIL":
        df_data_on_location = df_brasil.copy()
    else:
        df_data_on_location = df_states[(df_states["estado"] == location)]
    fig2 = go.Figure(layout={"template":"plotly_white"})
    bar_plots = ["casosNovos", "obitosNovos"]

    if plot_type in bar_plots:
        fig2.add_trace(go.Bar(x=df_data_on_location["data"], y=df_data_on_location[plot_type]))
    else:
        fig2.add_trace(go.Scatter(x=df_data_on_location["data"], y=df_data_on_location[plot_type]))
    
    fig2.update_layout(
        paper_bgcolor="#D3D3D3",
        plot_bgcolor="#D3D3D3",
        autosize=True,
        margin=dict(l=10, r=10, b=10, t=10),
        )
    return fig2


@app.callback(
    Output("choropleth-map", "figure"), 
    [Input("date-picker", "date")]
)
def update_map(date):
    df_data_on_states = df_states[df_states["data"] == date]

    fig = px.choropleth_mapbox(df_data_on_states, locations="estado", geojson=brazil_states, 
        center={"lat": CENTER_LAT, "lon": CENTER_LON},  # https://www.google.com/maps/ -> right click -> get lat/lon
        zoom=4, color="casosAcumulado", color_continuous_scale="Ice", opacity=0.55,
        hover_data={"casosAcumulado": True, "casosNovos": True, "obitosNovos": True, "estado": False}
        )

    fig.update_layout(paper_bgcolor="#D3D3D3", mapbox_style="carto-positron", autosize=True,  #"open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor"
                    margin=go.layout.Margin(l=0, r=0, t=0, b=0), showlegend=False)
    return fig


@app.callback(
    Output("location-button", "children"),
    [Input("choropleth-map", "clickData"), Input("location-button", "n_clicks")]
)
def update_location(click_data, n_clicks):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if click_data is not None and changed_id != "location-button.n_clicks":
        state = click_data["points"][0]["location"]
        return "{}".format(state)
    
    else:
        return "BRASIL"


#### Finally the execution step. In the output, a link to the dashboard will be displayed:

In [ ]:
if __name__=='__main__':
    app.run_server(debug=False)